In [ ]:
"""
Prepare cav mean and cav std

"""

In [ ]:
import xarray as xr
import summer_paper.useful_functions as uf
from tqdm.notebook import tqdm

READ IN DATA

In [ ]:
nemo_run = 'ctrl94' # 'EPM026','EPM031', 'EPM034'
if nemo_run in ['ctrl94']:
    yy_start = 1982
    yy_end = 2013
    yy_ex = 2000
elif nemo_run in ['isf94','isfru94']:
    yy_start = 2014
    yy_end = 2100
    yy_ex = 2016
elif nemo_run == 'OPM026':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM031':
    yy_start = 1999
    yy_end = 2068

In [ ]:
if nemo_run in ['ctrl94','isf94','isfru94']:
    inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'
    inputpath_mask = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
else:
    inputpath_data='/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
    inputpath_mask = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    inputpath_profiles = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'


In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
# dIF, dGL
if nemo_run in ['ctrl94','isf94','isfru94']:
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')
else:
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')

nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)
file_isf_conc = file_conc_cut['isfdraft_conc']


In [ ]:
for tt in tqdm(range(yy_start, yy_end+1)): #continue at 1978

    # T and S extrapolated to ice draft depth
    if nemo_run in ['ctrl94','isf94','isfru94']:
        T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc')
    else:
        T_S_2D_isfdraft = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS.nc').sel(profile_domain=50,time=tt).squeeze().drop('profile_domain')

    n = 0
    for kisf in file_isf.Nisf:
        T_isf = uf.choose_isf(T_S_2D_isfdraft['theta_in'],isf_stack_mask, kisf)
        S_isf = uf.choose_isf(T_S_2D_isfdraft['salinity_in'],isf_stack_mask, kisf)
        isf_conc_kisf = uf.choose_isf(file_isf_conc,isf_stack_mask, kisf) 
        weight_kisf = isf_conc_kisf

        T_mean_cav = uf.weighted_mean(T_isf, 'mask_coord', weight_kisf).to_dataset(name='T_mean')
        S_mean_cav = uf.weighted_mean(S_isf, 'mask_coord', weight_kisf).to_dataset(name='S_mean')
        T_std_cav = uf.weighted_std(T_isf, 'mask_coord', weight_kisf).to_dataset(name='T_std')
        S_std_cav = uf.weighted_std(S_isf, 'mask_coord', weight_kisf).to_dataset(name='S_std')


        all_TS_isf = xr.merge([T_mean_cav,S_mean_cav,T_std_cav,S_std_cav])
        all_TS_isf_2D = all_TS_isf.where(T_isf).load()

        if n == 0:
            all_TS_all = all_TS_isf_2D.squeeze().drop('Nisf')
        else:
             all_TS_all =  all_TS_all.combine_first(all_TS_isf_2D).squeeze().drop('Nisf')
        n = n+1

        del all_TS_isf_2D

    all_TS_all_unstacked = uf.bring_back_to_2D(all_TS_all)
    
    all_TS_all_unstacked.to_netcdf(inputpath_profiles + 'T_S_2D_meanstd_isf_draft_oneFRIS_'+str(tt)+'.nc')

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_2D_meanstd_isf_draft_oneFRIS_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_2D_meanstd_isf_draft_oneFRIS.nc')

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS.nc')

In [ ]:
print('test')